In [ ]:
def discrete_colorscale(bvals, colors):
    """
    bvals - list of values bounding intervals/ranges of interest
    colors - list of rgb or hex colorcodes for values in [bvals[k], bvals[k+1]],0<=k < len(bvals)-1
    returns the plotly  discrete colorscale
    """
    if len(bvals) != len(colors)+1:
        raise ValueError('len(boundary values) should be equal to  len(colors)+1')
    bvals = sorted(bvals)     
    nvals = [(v-bvals[0])/(bvals[-1]-bvals[0]) for v in bvals]  #normalized values
    
    dcolorscale = [] #discrete colorscale
    for k in range(len(colors)):
        dcolorscale.extend([[nvals[k], colors[k]], [nvals[k+1], colors[k]]])
    return dcolorscale    



def create_visualization(data_df,user_id,week_num,data_summary_user_current):
    import numpy as np
    import pandas as pd
    import plotly.express as px
    import plotly.graph_objects as go
#     import plotly
    import pickle,random
    data_df=data_df.loc[data_df['Stressor'].notnull()]
    user_df=data_df.loc[data_df.user==user_id]
    with open('/home/jupyter/sneupane/MOODS/pickled_files/colors.pickle', 'rb') as handle:
        colors = pickle.load(handle)
    random.shuffle(colors)
    if len(user_df)>0:       
        
        with open('/home/jupyter/sneupane/MOODS/pickled_files/stressor_abbr.pickle', 'rb') as handle:
            stressor_lemma = pickle.load(handle)
        
        y_modified_labels=stressor_lemma[user_id]
        colors=['#ff9868', '#ff7c51', '#ff5c37', '#ff2b16', '#ec0000', '#d00000', '#b50000', '#9b0000', '#820000', '#690000']

        bvals = [0,10,20,30,40,50,60,70,80,90,100]
        bvals=[i/100 for i in bvals ]
        dcolorsc = discrete_colorscale(bvals, colors)
        fig = go.Figure()
        stressors=user_df.Stressor.unique()
        user_df_likelihood=user_df.groupby(["Stressor"],as_index=False)["value"].mean().sort_values(["value"],ascending=True)
        user_df_likelihood=user_df_likelihood.round(3)
        user_df_likelihood["Stressor_abr"]=user_df_likelihood["Stressor"].map(y_modified_labels)
        user_df_likelihood["value"]=round(user_df_likelihood["value"],2)
#         user_df_likelihood["value_str"]=user_df_likelihood["value"].astype("str")
        user_df_likelihood["value_str"]=user_df_likelihood['value'].apply(lambda x: str(x)[1:])
        fig.add_trace(go.Bar(x=user_df_likelihood["value"],y=user_df_likelihood["Stressor_abr"],orientation='h',
                             text=user_df_likelihood["value_str"],name="",
                             customdata=user_df_likelihood[["Stressor","value_str"]], marker={'color': user_df_likelihood["value"],
                                                   'colorscale': dcolorsc},
                                             hovertemplate=
                                        "Stressor: %{customdata[0]}<br>" +
                                        "Stress Probability: %{customdata[1]}<br>" ,     

                           textposition="outside",
                            textfont=dict(
                                    family="sans serif",
                                    size=50,
                                   )))
        fig.update_layout(bargap=0.7)
        
        fig.update_layout(
                    autosize=True,legend_title="<b>Daily Stressors</b>",legend_title_side="top",

                    margin=dict(
                        l=20,
                        r=20,
                        b=20,
                        t=40,
                       
                    ),
        title={"text":"",
            "x":0.5,
             "yanchor":"top",
        "xanchor":"center",
            
           
        })
     
        fig.update_layout(hoverlabel=dict(
                                    bgcolor="white",
                                    font_size=18,
                                    font_family="Rockwell",
                                    bordercolor="Black"
                                ),
                          font=dict(
        family="Courier New, monospace",
        size=12,
        color="Black"
    ))
        fig.update_xaxes(range=[0,1.1])
        data_summary_user=data_summary_user_current["Stressor Ranking"]

        return "Stressor Ranking",fig,data_summary_user
    else:
        return "","",""